In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

df=pd.read_csv('/content/drive/MyDrive/EWS/Twitter/LABEL TWEETS/twitter_data_labeled/phase2_1.csv')

df.head()

df.columns

tweets=df.tweet.to_list()

tweets_train, tweets_validation = train_test_split(tweets,test_size=0.1,random_state=8)

def save_txt(texts,name):
    with open(name, 'w') as f:
        for text in texts:
            text = text if text[-1]=='/n' else text+'/n'
            f.write(text)

save_txt(tweets_train,'tweet_train.txt')

save_txt(tweets_validation,'tweets_validation.txt')

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# New Section

In [ ]:
!ls

drive  sample_data  tweets_validation.txt  tweet_train.txt


In [ ]:
!git clone https://github.com/huggingface/transformers

Cloning into 'transformers'...
remote: Enumerating objects: 88986, done.
remote: Counting objects: 100% (208/208), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 88986 (delta 98), reused 171 (delta 81), pack-reused 88778
Receiving objects: 100% (88986/88986), 71.82 MiB | 22.00 MiB/s, done.
Resolving deltas: 100% (64090/64090), done.


In [ ]:
!pwd

/content


In [ ]:
!pip install datasets==1.15.1

     |████████████████████████████████| 290 kB 5.3 MB/s 
     |████████████████████████████████| 59 kB 6.9 MB/s 
     |████████████████████████████████| 132 kB 45.0 MB/s 
     |████████████████████████████████| 1.1 MB 40.4 MB/s 
     |████████████████████████████████| 243 kB 44.9 MB/s 
     |████████████████████████████████| 271 kB 44.0 MB/s 
     |████████████████████████████████| 192 kB 44.2 MB/s 
     |████████████████████████████████| 160 kB 39.9 MB/s 


In [ ]:
# !pip uninstall transformers 

Found existing installation: transformers 4.13.0.dev0
Can't uninstall 'transformers'. No files were found to uninstall.


In [ ]:
!pip install transformers==4.12.3

     |████████████████████████████████| 3.1 MB 5.2 MB/s 
     |████████████████████████████████| 596 kB 48.6 MB/s 
     |████████████████████████████████| 3.3 MB 33.4 MB/s 
     |████████████████████████████████| 895 kB 28.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
%cd transformers

/content/transformers


In [ ]:
!pip install -e .

Obtaining file:///content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Attempting uninstall: transformers
    Found existing installation: transformers 4.13.0.dev0
    Can't uninstall 'transformers'. No files were found to uninstall.
  Running setup.py develop for transformers


In [ ]:
%cd ..

/content


In [ ]:
!python transformers/examples/pytorch/language-modeling/run_mlm.py \
    --model_name_or_path vinai/bertweet-base \
    --train_file /content/tweet_train.txt \
    --validation_file /content/tweets_validation.txt \
    --do_train \
    --do_eval \
    --output_dir='repretrainbertweet' \
    --model_type bert \
    --evaluation_strategy steps \
    --max_seq_length 128 \
    --per_device_train_batch_size 16 \
    --per_device_eval_batch_size 32 \
    --gradient_accumulation_steps 128 \
    --eval_accumulation_steps 10 \
    --learning_rate 1e-4 \
    --num_train_epochs 2 \
    --logging_steps 10 --eval_steps 10 --save_steps 10 \
    --save_total_limit 5 \
    --fp16 \
    --seed 20 \
    --line_by_line \
    --dataloader_num_workers 2 \
    --overwrite_output_dir

10/28/2021 19:07:23 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: True
10/28/2021 19:07:23 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=2,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=10,
eval_steps=10,
evaluation_strategy=IntervalStrategy.STEPS,
fp16=True,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=128,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=0.0001,
length_column_name=length,
load_best_model_at

In [ ]:
# !mv repretrainbertweet /content/drive/MyDrive/EWS/


In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir runs

Acceso al modelo entrenado

Clasificaion

In [ ]:
!ls /content/drive/MyDrive/EWS/repretrainbertweet

added_tokens.json  config.json		    tokenizer_config.json
all_results.json   eval_results.json	    trainer_state.json
bpe.codes	   pytorch_model.bin	    training_args.bin
checkpoint-10	   README.md		    train_results.json
checkpoint-20	   runs			    vocab.txt
checkpoint-30	   special_tokens_map.json


In [ ]:
!pip install transformers

In [ ]:
# model_name='vinai/bertweet-base'
# model_name='bert-base-cased'
model_name='/content/drive/MyDrive/EWS/repretrainbertweet'

In [ ]:
import transformers
from transformers import TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer,Trainer, DataCollatorWithPadding, default_data_collator, EarlyStoppingCallback
from datasets import load_dataset, DatasetDict 

In [ ]:
dataset=load_dataset('csv',data_files={'train':'/content/drive/MyDrive/EWS/Twitter/LABEL TWEETS/twitter_data_labeled/phase2_1.csv'})

Using custom data configuration default-a157e4b6914d4887


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-a157e4b6914d4887/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['ID', 'date', 'tweet', 'brand', 'relevance', 'label', 'check', 'flag'],
        num_rows: 71316
    })
})

In [ ]:
dataset['train'][1]['label']

1.0

In [ ]:
set([dataset['train'][i]['label'] for i in range(len(dataset['train']))])

{0.0, 1.0, 2.0, None}

In [ ]:
dataset=dataset.filter(lambda x: x['label'] is not None)

  0%|          | 0/72 [00:00<?, ?ba/s]

In [ ]:
# def change_labels(example):
#   if example['label'] is not None:
#     example['label']=int(example['label'])

In [ ]:
# [row['tweet'] for row in dataset['train'] if not isinstance(row['tweet'],str)]

In [ ]:
dataset_train_test=dataset['train'].train_test_split(test_size=0.1,seed=42)

In [ ]:
dataset_train_val=dataset_train_test['train'].train_test_split(test_size=0.12,seed=42)

In [ ]:
dataset=DatasetDict(
    {
        'train':dataset_train_val['train'],
        'validation':dataset_train_val['test'],
        'test': dataset_train_test['test']
    }
)

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['ID', 'date', 'tweet', 'brand', 'relevance', 'label', 'check', 'flag'],
        num_rows: 2782
    })
    validation: Dataset({
        features: ['ID', 'date', 'tweet', 'brand', 'relevance', 'label', 'check', 'flag'],
        num_rows: 380
    })
    test: Dataset({
        features: ['ID', 'date', 'tweet', 'brand', 'relevance', 'label', 'check', 'flag'],
        num_rows: 352
    })
})

In [ ]:
# tokenizer=AutoTokenizer.from_pretrained('/content/drive/MyDrive/EWS/repretrainbertweet',use_fast=True)
tokenizer=AutoTokenizer.from_pretrained(model_name,use_fast=True)

emoji is not installed, thus not converting emoticons or emojis into text. Please install emoji: pip3 install emoji


In [ ]:
def process_data(examples):
  tokenized=tokenizer(examples['tweet'],truncation=True)#padding=True)#,max_length=130)
  tokenized['label']=[int(label) for label in examples['label']] 
  return tokenized

In [ ]:
set(dataset['train'][:]['label'])

{0.0, 1.0, 2.0}

In [ ]:
# dataset['train'][20]['tweet']

In [ ]:
import transformers
transformers.__version__

'4.12.3'

In [ ]:
# dataset=load_dataset('glue','cola')

In [ ]:
dataset=dataset.map(process_data,batched=True,remove_columns=dataset['train'].column_names,load_from_cache_file=False)

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'token_type_ids'],
        num_rows: 2782
    })
    validation: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'token_type_ids'],
        num_rows: 380
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'token_type_ids'],
        num_rows: 352
    })
})

In [ ]:
dataset['train'][1]['label']

1.0

In [ ]:
columns_to_return = ['input_ids', 'labels', 'attention_mask']
# dataset.set_format(type='torch', columns=columns_to_return)


In [ ]:
args=TrainingArguments(
    output_dir='/content/drive/MyDrive/EWS/Class_Model',
    per_device_train_batch_size=32,
    per_device_eval_batch_size=128,
    gradient_accumulation_steps=2,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    # save_steps=200,
    # logging_steps=200,
    # eval_steps=200,
    fp16 = True,
    metric_for_best_model='eval_f1-score',
    greater_is_better=True,
    num_train_epochs=10,
    save_total_limit=5,
    load_best_model_at_end=True
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
from sklearn.metrics import classification_report

In [ ]:
def compute_metrics(pred):
  preds,labels=pred.predictions,pred.label_ids
  preds=np.argmax(preds,axis=1)
  return classification_report(labels,preds,output_dict=True)['macro avg']
  # import fbeta_score from
  #{f1-score:fbeta_score(labels,preds,2)
  #probar esto

In [ ]:
# data_collator=DataCollatorWithPadding(tokenizer, padding='longest',max_length=512)

In [ ]:
# dataset
# for split in ['train','validation','test']:
#   for i in range(len(dataset[split])):
#     dataset[split][i]['label']=int(dataset[split][i]['label'])

In [ ]:
lab_to_id={l:i for i,l in enumerate(set(dataset['train'][:]['label']))}
def _map(example):
  example['labels']=lab_to_id[example['label']]
  return example
dataset=dataset.map(_map,remove_columns=['label'])

  0%|          | 0/2782 [00:00<?, ?ex/s]

  0%|          | 0/380 [00:00<?, ?ex/s]

  0%|          | 0/352 [00:00<?, ?ex/s]

In [ ]:
type(dataset['train'][0]['labels'])

int

In [ ]:
import numpy as np

In [ ]:
trainer=Trainer(model=AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3), #('/content/drive/MyDrive/EWS/repretrainbertweet',),
                args=args,
                train_dataset=dataset['train'],
                eval_dataset=dataset['validation'],
                tokenizer=tokenizer,
                compute_metrics=compute_metrics,
                callbacks=[EarlyStoppingCallback(1,0.0001)]
                # data_collator=None
                )

loading configuration file /content/drive/MyDrive/EWS/repretrainbertweet/config.json
Model config RobertaConfig {
  "_name_or_path": "vinai/bertweet-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 130,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertweetTokenizer",
  "torch_dtype": "float32",
  "transformers_version": "4.12.3",
  "type_vocab_size": 1,
  "use_cache": true,
  

In [ ]:
set(dataset['train'][:]['labels'])

{0, 1, 2}

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 2782
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 2
  Total optimization steps = 430


Epoch,Training Loss,Validation Loss,Precision,Recall,F1-score,Support
0,No log,0.380571,0.470588,0.528627,0.497021,380
1,No log,0.336007,0.809091,0.765397,0.782654,380
2,No log,0.325770,0.839785,0.768457,0.786080,380
3,No log,0.366954,0.823079,0.781012,0.798814,380
4,No log,0.318016,0.848027,0.783303,0.810038,380
5,No log,0.410919,0.841949,0.787807,0.811831,380
6,No log,0.404584,0.824066,0.806970,0.814997,380
7,No log,0.471158,0.828471,0.786285,0.794837,380


***** Running Evaluation *****
  Num examples = 380
  Batch size = 128
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to /content/drive/MyDrive/EWS/Class_Model/checkpoint-43
Configuration saved in /content/drive/MyDrive/EWS/Class_Model/checkpoint-43/config.json
Model weights saved in /content/drive/MyDrive/EWS/Class_Model/checkpoint-43/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/EWS/Class_Model/checkpoint-43/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/EWS/Class_Model/checkpoint-43/special_tokens_map.json
added tokens file saved in /content/drive/MyDrive/EWS/Class_Model/checkpoint-43/added_tokens.json
Deleting older checkpoint [/content/drive/MyDr

TrainOutput(global_step=344, training_loss=0.17221918771433276, metrics={'train_runtime': 1699.0966, 'train_samples_per_second': 16.373, 'train_steps_per_second': 0.253, 'total_flos': 940891526221752.0, 'train_loss': 0.17221918771433276, 'epoch': 7.99})

In [ ]:
# ! transformers-cli env
